In [2]:
import os
import csv
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_b0
from tqdm import tqdm

from data_loading import TorchTensorFolderDataset

In [3]:
def save_to_csv(data, filename, column_names):
    timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H.%M")
    filename = f"{filename}_{timestamp}.csv"
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  
        writer.writerows(data)

    return

In [4]:
def get_loader(data_size:str="full", target_data:str="train", batch_size:int=16):
    # path_to_raw = os.path.join(os.getcwd(), "data", "preprocessed", data_size, "raw", target_data)
    path_to_mel = os.path.join(os.getcwd(), "data", "preprocessed", data_size, "mel", target_data)
    
    # dataset = EnsembleDataset(path_to_raw, path_to_mel)
    dataset = TorchTensorFolderDataset(path_to_mel)
    # n = len(dataset.labels)
    n = len(dataset.class_to_idx)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return loader, n

In [5]:
DATA_SIZE = ""

In [6]:
def get_pretrained_model():
    model = efficientnet_b0(weights='EfficientNet_B0_Weights.IMAGENET1K_V1')
    
    original_conv = model.features[0][0]
    new_conv = nn.Conv2d(
        in_channels=1,
        out_channels=original_conv.out_channels,
        kernel_size=original_conv.kernel_size,
        stride=original_conv.stride,
        padding=original_conv.padding,
        bias=original_conv.bias is not None
    )
    with torch.no_grad():
        new_conv.weight[:] = original_conv.weight.mean(dim=1, keepdim=True)
    model.features[0][0] = new_conv
    
    return model

In [7]:
def train_model(model, optimizer, train_loader, val_loader, num_epochs=5, patience=3, verbose=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    model.to(device)

    counter = 0
    best_val_acc = 0
    best_model = model.state_dict()

    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        if not patience is None and counter >= patience:
            print('Patience trigger. End of learning.')
            break
            
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        train_acc = 100 * train_correct / train_total
    
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
    
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_acc = 100 * val_correct / val_total

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if verbose:
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            print(f"Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")
            print(f"Val Loss:   {val_loss/len(val_loader):.4f}, Val Acc:   {val_acc:.2f}%\n")

        if val_acc > best_val_acc:
            counter = 0
            best_val_acc = val_acc
            best_model = model.state_dict()
        else:
            counter += 1

    model.load_state_dict(best_model)

    return train_losses, val_losses

In [8]:
def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    
    model.eval()
    test_loss = 0
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
    
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    test_acc = 100 * test_correct / test_total

    return test_loss, test_acc

In [9]:
def test_learning_rates(times=3, learning_rates=[0.1, 0.05, 0.01, 0.005, 0.001]):
    val_loader, _ = get_loader(DATA_SIZE, "validation")
    test_loader, _ = get_loader(DATA_SIZE, "test")
    
    result = []
    column_names = ["learning_rate", "train_loss", "train_acc", "val_loss", "val_acc", "test_loss", "test_acc"]
    print(*column_names)
    
    for learning_rate in learning_rates:
        for _ in range(times):
            train_loader, n = get_loader(DATA_SIZE, "train")
            
            model = get_pretrained_model()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            
            train_model(model, optimizer, train_loader, val_loader, num_epochs=10)
            
            train_loss, train_acc = test_model(model, train_loader)
            val_loss, val_acc = test_model(model, val_loader)
            test_loss, test_acc = test_model(model, test_loader)
            
            result.append((learning_rate, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))
            
            print(*result[-1])
            
    save_to_csv("results/learning_rates_test.csv", column_names, result)

In [10]:
def test_batch_sizes(times=3, batch_sizes=[16, 32, 64]):
    val_loader, _ = get_loader(DATA_SIZE, "validation")
    test_loader, _ = get_loader(DATA_SIZE, "test")
    
    result = []
    column_names = ["batch_size", "train_loss", "train_acc", "val_loss", "val_acc", "test_loss", "test_acc"]
    print(*column_names)
    
    for batch_size in batch_sizes:
        for _ in range(times):
            train_loader, n = get_loader(DATA_SIZE, "train", batch_size)
            
            model = get_pretrained_model()
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            
            train_model(model, optimizer, train_loader, val_loader, num_epochs=10)
            
            train_loss, train_acc = test_model(model, train_loader)
            val_loss, val_acc = test_model(model, val_loader)
            test_loss, test_acc = test_model(model, test_loader)
            
            result.append((batch_size, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))
            
            print(*result[-1])
            
    save_to_csv("results/batch_sizes_test.csv", column_names, result)

In [10]:
train_loader, n = get_loader(DATA_SIZE, "train", batch_size=16)
val_loader, _ = get_loader(DATA_SIZE, "validation")
model = get_pretrained_model()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
train_model(model, optimizer, train_loader, val_loader, verbose=True)

Epoch [1/5]
Train Loss: 1.5983, Train Acc: 60.70%
Val Loss:   0.3912, Val Acc:   88.45%



Epoch [2/5]
Train Loss: 0.3249, Train Acc: 90.27%
Val Loss:   0.2586, Val Acc:   92.04%



Epoch [3/5]
Train Loss: 0.2093, Train Acc: 93.76%
Val Loss:   0.2205, Val Acc:   93.65%



Epoch [4/5]
Train Loss: 0.1537, Train Acc: 95.44%
Val Loss:   0.1978, Val Acc:   94.16%



Epoch [5/5]
Train Loss: 0.1233, Train Acc: 96.26%
Val Loss:   0.2277, Val Acc:   93.51%



([5103.514760758728,
  1037.3955498253927,
  668.3299468760379,
  490.8038711771369,
  393.6687472044141],
 [166.26543576689437,
  109.90196986513911,
  93.72935182449874,
  84.05303023275337,
  96.77544712425151])

In [11]:
train_loader, n = get_loader(DATA_SIZE, "train", batch_size=16)
val_loader, _ = get_loader(DATA_SIZE, "validation")
model = get_pretrained_model()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, optimizer, train_loader, val_loader, verbose=True)

Epoch [1/5]
Train Loss: 0.6204, Train Acc: 83.08%
Val Loss:   0.3020, Val Acc:   91.67%



Epoch [2/5]
Train Loss: 0.2388, Train Acc: 93.07%
Val Loss:   0.2097, Val Acc:   93.70%



Epoch [3/5]
Train Loss: 0.1853, Train Acc: 94.70%
Val Loss:   0.1769, Val Acc:   94.79%



Epoch [4/5]
Train Loss: 0.1577, Train Acc: 95.40%
Val Loss:   0.2058, Val Acc:   94.62%



Epoch [5/5]
Train Loss: 0.1355, Train Acc: 96.06%
Val Loss:   0.1846, Val Acc:   94.85%



([1980.8902250328101,
  762.5810027393163,
  591.5424284026376,
  503.47922662697965,
  432.75208029104397],
 [128.33013684261823,
  89.142952112481,
  75.19214589620242,
  87.47840556570736,
  78.44605588639388])

In [12]:
def get_loader(data_size:str="full", target_data:str="train", batch_size:int=16):
    # path_to_raw = os.path.join(os.getcwd(), "data", "preprocessed", data_size, "raw", target_data)
    path_to_mel = os.path.join(os.getcwd(), "data", "preprocessed","denoised", data_size, "mel", target_data)
    
    # dataset = EnsembleDataset(path_to_raw, path_to_mel)
    dataset = TorchTensorFolderDataset(path_to_mel)
    # n = len(dataset.labels)
    n = len(dataset.class_to_idx)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=False)

    return loader, n

train_loader, n = get_loader('', "train", batch_size=16)
val_loader, _ = get_loader('', "validation")
model = get_pretrained_model()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, optimizer, train_loader, val_loader, verbose=True, num_epochs=10)

Epoch [1/10]
Train Loss: 0.6007, Train Acc: 83.57%
Val Loss:   0.2776, Val Acc:   91.89%



Epoch [2/10]
Train Loss: 0.2418, Train Acc: 92.98%
Val Loss:   0.2813, Val Acc:   91.97%



Epoch [3/10]
Train Loss: 0.1909, Train Acc: 94.45%
Val Loss:   0.2228, Val Acc:   93.29%



Epoch [4/10]
Train Loss: 0.1606, Train Acc: 95.34%
Val Loss:   0.1979, Val Acc:   94.42%



Epoch [5/10]
Train Loss: 0.1354, Train Acc: 96.00%
Val Loss:   0.1989, Val Acc:   94.19%



...

Epoch [7/10]
Train Loss: 0.1120, Train Acc: 96.70%
Val Loss:   0.1839, Val Acc:   95.13%



Epoch [8/10]
Train Loss: 0.1032, Train Acc: 97.01%
Val Loss:   0.1891, Val Acc:   95.15%



Epoch [9/10]
Train Loss: 0.0975, Train Acc: 97.16%
Val Loss:   0.1811, Val Acc:   94.60%



Epoch [10/10]
Train Loss: 0.0881, Train Acc: 97.44%
Val Loss:   0.2028, Val Acc:   94.82%



([1918.0213053759653,
  772.0496782499831,
  609.4463884253637,
  512.8738083560165,
  432.27412884963996,
  394.1068180496659,
  357.5698853743961,
  329.3669393299715,
  311.3240869319525,
  281.4477150152161],
 [117.98939325893298,
  119.55551666021347,
  94.6768495227152,
  84.12303739748313,
  84.53763370207889,
  83.70196658827626,
  78.17026244670888,
  80.38090721858316,
  76.98737024701586,
  86.20373453723096])